# Integration Test 2

In [ ]:
# Imports
import numpy as np
import scipy
import scipy.signal as signal
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.interpolate import CubicSpline
from scipy.signal import hilbert
import numpy as np
import ipywidgets as ipw
import base64
from random import randint
from pynq import Clocks
import xrfdc
import os
from pynq.lib import Pmod_IO
import time

# Use the RFSoC base overlay
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

# Start RF clocks
base.init_rf_clks()

# ADC Initialization
Use ADC D for this demo. Both mixers bypassed, real data viewed. Sample frequency is 4.9152GHz. Decimator is bypassed.

In [ ]:
# ADC initialization
# Channels
DAC_CHANNEL_B = 0 # 'Channel 0': {'Tile': 224, 'Block': 0}
DAC_CHANNEL_A = 1 # 'Channel 1': {'Tile': 230, 'Block': 0}

ADC_CHANNEL_D = 0 # 'Channel 0': {'Tile': 224, 'Block': 0}
ADC_CHANNEL_C = 1 # 'Channel 1': {'Tile': 224, 'Block': 1}
ADC_CHANNEL_B = 2 # 'Channel 2': {'Tile': 226, 'Block': 0}
ADC_CHANNEL_A = 3 # 'Channel 3': {'Tile': 226, 'Block': 1}

adc_array = [ADC_CHANNEL_C, ADC_CHANNEL_A]

adc_char_array = ['D', 'C', 'B', 'A']

number_samples = int(32768/2048)  # Between 16 and 32768
decimation_factor = 1 # 2 is default
sample_frequency = 4915.2e6/decimation_factor  # Hz The default sample frequency is 4915.2e6 Hz which is sufficient for our signal

original_adc_settings = base.radio.receiver.channel[ADC_CHANNEL_D].adc_block.MixerSettings

for ADC in adc_array:
    base.radio.receiver.channel[ADC].adc_block.DecimationFactor = decimation_factor
    base.radio.receiver.channel[ADC].adc_block.MixerSettings = {
        'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
        'EventSource':    xrfdc.EVNT_SRC_TILE, 
        'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
        'Freq':           1240.0,
        'MixerMode':      xrfdc.MIXER_MODE_R2C,
        'MixerType':      xrfdc.MIXER_TYPE_FINE,
        'PhaseOffset':    0.0
    }
    base.radio.receiver.channel[ADC].adc_block.UpdateEvent(xrfdc.EVENT_MIXER)
    
print("Original ADC settings:", original_adc_settings)
print("New ADC settings:", base.radio.receiver.channel[ADC].adc_block.MixerSettings)

# Initialize PMOD
Use one out pin to trigger Lazy Harold to perform one twitch. Drive pin low to start.

In [ ]:
lazy_harold_ear = Pmod_IO(base.PMODB,2,'out')

lazy_harold_ear.write(0)    # Drive pin low

one = 0
zero = 0
count = 0
while count < 100000:
    if lazy_harold_mouth.read() == 1:
        one += 1
    else:
        zero += 1
    count += 1
print("one:", one)
print("zero:", zero)
print(one/(zero+one)*100)
    

# View raw and interpolated data
Print power of each signal and values used in calculation for comparison with scope values.

# Perform Measure and Twitch Sequence
UI controls step size and range of sweep. Test [-90, 90] degrees at step size of .9 degrees for this demo.
Expecting 201 steps and 201 corresponding, calculated power values. Expecting max power of 10.44009951 dBm. Successful if max power is between 4 and 10 dBm.

In [ ]:
lazy_harold_ear.write(0)    # Drive pin low

# Testing

In [ ]:

# Phased array variables
d = 0.5 # half wavelength spacing
Nr = len(adc_array)

# Choose steering angle
steering_angle_degrees = 10
steering_angle = steering_angle_degrees / 180 * np.pi # convert to radians

# Calculate beamforming weights
beamforming_weights = np.exp(-2j * np.pi * d * np.arange(Nr) * np.sin(steering_angle)) # array factor
print("beamforming_weights:", beamforming_weights)

# Containers
rx_data = []
rx_real_data = []
raw_data = []
beamformed_data = []
interpolated_beamformed_data = []
summed_signal = []
final_data = []

# Create Plotly figure for interpolated ADC data
raw_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Needed for interpolation
time_data = np.arange(0, number_samples/sample_frequency, 1/sample_frequency)
dense_t = np.linspace(time_data.min(), time_data.max(), len(time_data) * 10)  # Increase density
    
# Get chunk of data, interpolate it, and create trace for each ADC in adc_array
for index, ADC in enumerate(adc_array):
    rx_data.append(base.radio.receiver.channel[ADC].transfer(number_samples))
    # print("rx_data:", rx_data)
    rx_real_data.append(np.real(rx_data[index]))
    sampled_signal = np.real(rx_data[index])
    cs_real = CubicSpline(time_data, sampled_signal)

    # Raw interpolated data
    raw_data.append(cs_real(dense_t))

    # # Add actual data trace
    # actual_fig.add_trace(
    # go.Scatter(x=time_data, y=sampled_signal, name=f"Actual Data ADC {adc_char_array[ADC]}"),
    # secondary_y=False,
    # )

    # Add raw interpolated data trace
    raw_fig.add_trace(
    go.Scatter(x=dense_t, y=raw_data[index], name=f"Interpolated Data ADC {adc_char_array[ADC]}"),
    secondary_y=False,
    )

# Update layout of raw ADC data
raw_fig.update_layout(
title=f"Time Domain Plot of ADC Channels",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show raw ADC data
raw_fig.show()    

# Calculate max power of each ADC channel
for index, ADC in enumerate(adc_array):
    max_amplitude = rx_real_data[index][np.argmax(rx_data[index])]
    max_power = 10*np.log10((np.square(max_amplitude/np.sqrt(2))/100)/.001)
    print("Max power of ADC", adc_char_array[ADC], ":", max_power, "dBm.")
    
# Create Plotly figure for interpolated ADC data
beamformed_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Shape interpolated_data into 2xnumber_samples
rx_data_reshaped = np.reshape(rx_data, (len(adc_array), number_samples))

# Shape beamforming_weights into 2xlen(adc_array)
beamforming_weights_reshaped = np.reshape(beamforming_weights, (-1, 1))

# Perform matrix multiplication
beamformed_data = beamforming_weights_reshaped * rx_data_reshaped

# Size should be len(adc_array)xnumber_samples
beamformed_data = np.asarray(beamformed_data).squeeze().real

# Add phase shifted traces to beamform_fig
for index, ADC in enumerate(adc_array):
    # Add interpolated beamformed data trace
    sampled_signal = beamformed_data[index]
    cs_real = CubicSpline(time_data, sampled_signal)

    # Interpolate beamformed data
    interpolated_beamformed_data.append(cs_real(dense_t))

    # Add interpolated data trace
    beamformed_fig.add_trace(
    go.Scatter(x=dense_t, y=interpolated_beamformed_data[index], name=f"Beamformed Data ADC {adc_char_array[ADC]}"),
    secondary_y=False,
    )
    
# Update layout of beamform_fig
beamformed_fig.update_layout(
title=f"Time Domain Plot of Beamformed ADC Data",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show ADC channels after phase shifts
beamformed_fig.show()
    
# Create Plotly figure for final summed signal
final_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Apply weights and output 1x320
summed_signal = beamforming_weights.conj().T @ rx_data
summed_signal = np.asarray(summed_signal).squeeze().real

# Add summed data trace
sampled_signal = summed_signal
cs_real = CubicSpline(time_data, sampled_signal)

# Interpolate beamformed data
final_data.append(cs_real(dense_t))

# Add interpolated data trace
final_fig.add_trace(
go.Scatter(x=dense_t, y=final_data[0], name=f"Final Data {adc_char_array[ADC]}"),
secondary_y=False,
)
    
# Update layout of final_fig
final_fig.update_layout(
title=f"Time Domain Plot of Final Summed Signal",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show final signal
final_fig.show()

# Calculate power of final summed signal
power_data = 10*np.log10((np.square(summed_signal[np.argmax(summed_signal)]/np.sqrt(2))/100)/.001)

print("Power of Summed Signal:", power_data)

In [ ]:

# Beamforming variables
d = 0.5 # half wavelength spacing
Nr = len(adc_array)
steering_angle_degrees = 0 # direction of arrival (feel free to change this, it's arbitrary)
steering_angle = steering_angle_degrees / 180 * np.pi # convert to radians
beamforming_weights = np.exp(-2j * np.pi * d * np.arange(Nr) * np.sin(steering_angle)) # array factor
print("beamforming_weights:", beamforming_weights)
beamformed_data = [None] * len(adc_array)
summed_signal = []

# Test system variables
message = 1
LH_step_size = .9
start_angle = -90 # (degrees)
sweep_degrees = 180 # Choose MUST BE MULTIPLE OF .9 (degrees)
if (int(sweep_degrees%LH_step_size)) != 0:
    print("Invalid sweep_degrees. Choose value divisible by", LH_step_size)
number_of_measurements = (sweep_degrees / LH_step_size) + 1
end_angle = start_angle + sweep_degrees # (degrees)
actual_radians = sweep_degrees*(np.pi/180)
# print("Sweeping", sweep_degrees, "degrees to generate radiation pattern for phased array with", Nr, "elements at steering angle of", steering_angle_degrees, "using", int(number_of_measurements), "measurements recorded between", start_angle, "degrees and", end_angle, "degrees every", LH_step_size, "degrees...")

rx_amplitude_data = []
rx_data = []
power_data = []
summed_signal = []

# for measurement in range(0, int(number_of_measurements)):
    
# Take measurement
for ADC in adc_array:
    rx_amplitude_data.append(np.real(base.radio.receiver.channel[ADC].transfer(number_samples)))

beamformed_data = beamforming_weights.conj().T @ rx_amplitude_data
print("beamformed_data:", beamformed_data)
print("length of beamformed_data:", len(beamformed_data))
summed_signal = np.asarray(beamformed_data).squeeze().real
power_data.append(10*np.log10((np.square(summed_signal[np.argmax(summed_signal)]/np.sqrt(2))/100)/.001))
rx_amplitude_data = []

# Tell LH measurement is complete
# if measurement != (number_of_measurements - 1):
#     lazy_harold_ear.write(message)
#     if message == 1:
#         message = 0
#     else:
#         message = 1

    # Give LH time to complete a twitch
    # time.sleep(1)  
        
# fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
# ax.plot(np.linspace(-np.pi/2, (-np.pi/2)+actual_radians, int(number_of_measurements)), power_data) # MAKE SURE TO USE RADIAN FOR POLAR
# ax.set_steering_angle_zero_location('N') # make 0 degrees point up
# ax.set_steering_angle_direction(-1) # increase clockwise
# ax.set_rlabel_position(55)  # Move grid labels away from other labels
# plt.show()



print("summed_signal:", summed_signal)
print("power_data:", power_data)
# print("Expected amount of steps and measurements:", 201)
# print("Actual amount of steps and measurements:", len(summed_signal))
# print("Expected max power:", 10.44009951, "dBm")
# print("Actual max power:", summed_signal[np.argmax(summed_signal)], "dBm")
# print("Detected at:", start_angle + (np.argmax(summed_signal)*LH_step_size), "degrees")
# print("Average power:", np.average(summed_signal), "dBm")